In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session




/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


In [92]:
## Read Data

train = pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')
test = pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')

train = train.dropna()
#test = test.dropna()

## Transform The Data ( Both Test And Train)

from sklearn import preprocessing
from sklearn import preprocessing
  
# label_encoder object knows 
label_encoder = preprocessing.LabelEncoder()
  
train['CryoSleep']= label_encoder.fit_transform(train['CryoSleep'])
train['VIP']= label_encoder.fit_transform(train['VIP'])
train['Transported'] = label_encoder.fit_transform(train['Transported'])
train['Destination'] = label_encoder.fit_transform(train['Destination'])


test['CryoSleep']= label_encoder.fit_transform(test['CryoSleep'])
test['VIP']= label_encoder.fit_transform(test['VIP'])
test['Destination'] = label_encoder.fit_transform(test['Destination'])



In [3]:
train.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,0,B/0/P,2,39.0,0,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,0
1,0002_01,Earth,0,F/0/S,2,24.0,0,109.0,9.0,25.0,549.0,44.0,Juanna Vines,1
2,0003_01,Europa,0,A/0/S,2,58.0,1,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,0
3,0003_02,Europa,0,A/0/S,2,33.0,0,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,0
4,0004_01,Earth,0,F/1/S,2,16.0,0,303.0,70.0,151.0,565.0,2.0,Willy Santantines,1


In [48]:
train.corr()[9:10]  

/tmp/ipykernel_32/706965782.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  train.corr()[9:10]


,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
Transported,0.462803,-0.123773,-0.082553,-0.04226,-0.247291,0.055025,0.011602,-0.219854,-0.20795,1.0


**Taking the Input & Splitting** 

In [28]:
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics

X = train[['CryoSleep','Destination','Age','VIP','RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']]
X = X.values
y = train[['Transported']]
y = y.values

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15 ,random_state=1)

# x1_test = test[['CryoSleep','Destination','Age','VIP','RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']]
# x1_test = x1_test.values

import xgboost as xgb

# Create regression matrices
dtrain_reg = xgb.DMatrix(X_train, y_train, enable_categorical=True)
dtest_reg = xgb.DMatrix(X_test, y_test, enable_categorical=True)

**Best Fold Selection for cross validation**

In [42]:
import sklearn
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

xgb_model = xgb.XGBClassifier()
optimization_dict = {'max_depth': [2,4,6],
                     'n_estimators': [50,100,200]}

model = GridSearchCV(xgb_model, optimization_dict, 
                     scoring='accuracy', verbose=1)

model.fit(X_train,y_train)
print(model.best_score_)
print(model.best_params_)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
0.7939447907390916
{'max_depth': 2, 'n_estimators': 50}


****Training the Model****

In [46]:
# Define hyperparameters
params = {"objective": "binary:hinge", "eval_metric": "logloss"}
evals = [(dtrain_reg, "train"), (dtest_reg, "validation")]
n = 50
model = xgb.train(
   params=params,
   dtrain=dtrain_reg,
   evals = evals,
   verbose_eval=50,
   early_stopping_rounds=50,
   num_boost_round=n,
)

[0]	train-logloss:18.29274	validation-logloss:18.25339
[49]	train-logloss:6.14788	validation-logloss:7.65824


In [47]:
preds = model.predict(dtest_reg)

In [48]:
from sklearn import metrics
from sklearn import tree
print("Accuracy:",metrics.accuracy_score(y_test, preds))

Accuracy: 0.7921291624621595


In [49]:
params = {"objective": "binary:hinge", "eval_metric": "logloss"}
n = 50

results = xgb.cv(
   params, dtrain_reg,
   num_boost_round=n,
   nfold=2,
   early_stopping_rounds=20
)

In [44]:
results.tail(5)

,train-logloss-mean,train-logloss-std,test-logloss-mean,test-logloss-std
12,6.384064,0.123527,7.886671,0.342589
13,6.338128,0.169463,7.899792,0.329469
14,6.311887,0.143223,7.873540,0.290097
15,6.193785,0.143244,7.906345,0.283541
16,6.187230,0.110439,7.794798,0.250715


**Final Prediction**

Age,Roomservice,FoodCourt, ShoppingMall,Spa,VRDeck

In [126]:
X_TEST = test[['CryoSleep','Destination','Age','VIP','RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']]
mean_value_age =X_TEST['Age'].mean()
mean_value_RoomService =X_TEST['RoomService'].mean()
mean_value_FoodCourt =X_TEST['FoodCourt'].mean()
mean_value_ShoppingMall =X_TEST['ShoppingMall'].mean()
mean_value_Spa =X_TEST['Spa'].mean()
mean_value_VRDeck =X_TEST['VRDeck'].mean()

X_TEST['Age'].fillna(value=mean_value_age, inplace=True)
X_TEST['RoomService'].fillna(value=mean_value_RoomService, inplace=True)
X_TEST['FoodCourt'].fillna(value=mean_value_FoodCourt, inplace=True)
X_TEST['ShoppingMall'].fillna(value=mean_value_ShoppingMall, inplace=True)
X_TEST['Spa'].fillna(value=mean_value_Spa, inplace=True)
X_TEST['VRDeck'].fillna(value=mean_value_VRDeck, inplace=True)



X_TEST = X_TEST.values 

dtest_model = xgb.DMatrix(X_TEST, enable_categorical=True)
final_preds = model.predict(dtest_model)



/tmp/ipykernel_32/4167302845.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_TEST['Age'].fillna(value=mean_value_age, inplace=True)
/tmp/ipykernel_32/4167302845.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_TEST['RoomService'].fillna(value=mean_value_RoomService, inplace=True)
/tmp/ipykernel_32/4167302845.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_TEST['FoodCourt'].fillna(value=mean_value_FoodCourt, inplace=T

**Save the Output**

In [127]:
X_TEST = test[['PassengerId','CryoSleep','Destination','Age','VIP','RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']]
final_preds = pd.DataFrame(final_preds)
X_TEST['Transported'] = final_preds

submission = X_TEST[['PassengerId','Transported']]
submission['Transported'] = submission['Transported'].map({0:'False',1:'True'})



/tmp/ipykernel_32/1771489828.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_TEST['Transported'] = final_preds
/tmp/ipykernel_32/1771489828.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission['Transported'] = submission['Transported'].map({0:'False',1:'True'})


In [130]:
submission

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True
...,...,...
4272,9266_02,True
4273,9269_01,False
4274,9271_01,True
4275,9273_01,True


In [129]:
submission.fillna('False', inplace=True)

/tmp/ipykernel_32/2091112524.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission.fillna('False', inplace=True)


In [131]:
#submission.to_csv('/kaggle/working/submission.csv')

In [137]:
submission.reset

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True
...,...,...
4272,9266_02,True
4273,9269_01,False
4274,9271_01,True
4275,9273_01,True


In [138]:
df = submission.reset_index(drop=True)

In [139]:
df

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True
...,...,...
4272,9266_02,True
4273,9269_01,False
4274,9271_01,True
4275,9273_01,True
